#### Dataset Creation

In [1]:
from utils import set_seed
from data_loading.models_dataset import ArchiMateModelDataset

set_seed(42)

config_params = dict(
    min_enr = -1,
    min_edges = 10,
    language = 'en',
)
dataset_name = 'eamodelset'

dataset = ArchiMateModelDataset(dataset_name, **config_params)

Loading eamodelset from pickle
Loaded eamodelset with 558 graphs
Loaded eamodelset with 558 graphs
Graphs: 558


In [2]:
from data_loading.graph_dataset import GraphEdgeDataset, GraphNodeDataset

graph_data_params = dict(
    # reload=True,
    test_ratio=0.2,
    add_negative_train_samples=True,
    neg_sampling_ratio=1,
    distance=0,
    use_special_tokens=True,
    random_embed_dim=1,
    # use_edge_types=True,
    use_node_types=True,
    use_embeddings=True,
    # embed_model_name='bert-base-cased',
    ckpt='results/eamodelset/lp/10_att_0_nt_0/checkpoint-177600',
    limit = -1,
)

print("Loading graph dataset")
graph_edge_dataset = GraphEdgeDataset(dataset, **graph_data_params)
print("Loaded graph dataset")

Loading graph dataset
Using tokenizer
Loading tokenizer for bert-base-uncased


Embeddings graphs:   0%|          | 0/558 [00:00<?, ?it/s]

Creating graphs:   0%|          | 0/558 [00:00<?, ?it/s]

['AndJunction' 'ApplicationCollaboration' 'ApplicationComponent'
 'ApplicationEvent' 'ApplicationFunction' 'ApplicationInteraction'
 'ApplicationInterface' 'ApplicationProcess' 'ApplicationService'
 'Artifact' 'Assessment' 'BusinessActor' 'BusinessCollaboration'
 'BusinessEvent' 'BusinessFunction' 'BusinessInteraction'
 'BusinessInterface' 'BusinessObject' 'BusinessProcess' 'BusinessRole'
 'BusinessService' 'Capability' 'CommunicationNetwork' 'Constraint'
 'Contract' 'CourseOfAction' 'DataObject' 'Deliverable' 'Device'
 'DistributionNetwork' 'Driver' 'Equipment' 'Facility' 'Gap' 'Goal'
 'Grouping' 'ImplementationEvent' 'Junction' 'Location' 'Material'
 'Meaning' 'Node' 'OrJunction' 'Outcome' 'Path' 'Plateau' 'Principle'
 'Product' 'Representation' 'Requirement' 'Resource' 'Stakeholder'
 'SystemSoftware' 'TechnologyCollaboration' 'TechnologyEvent'
 'TechnologyFunction' 'TechnologyInteraction' 'TechnologyInterface'
 'TechnologyProcess' 'TechnologyService' 'Value' 'ValueStream'
 'WorkPack

Validating edge classes:   0%|          | 0/558 [00:00<?, ?it/s]

Train classes: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Test classes: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Number of classes in training set: 11
Number of classes in test set: 11
Train edge classes: {4: 12783, 8: 5362, 7: 7065, 3: 7522, 2: 5802, 1: 3550, 10: 4397, 9: 1205, 5: 4413, 6: 692, 0: 4011}
Test edge classes: {4: 3109, 7: 1808, 3: 1827, 8: 1395, 5: 1049, 10: 1100, 1: 848, 0: 956, 2: 1359, 9: 301, 6: 171}
Loaded graph dataset


In [ ]:
graph_edge_dataset.graphs[0].data, graph_edge_dataset[0]

(NumpyData(train_edge_mask=[40], test_edge_mask=[10], train_pos_edge_label_index=[2, 40], train_pos_edge_label=[40], train_neg_edge_label_index=[2, 40], train_neg_edge_label=[40], test_pos_edge_label_index=[2, 10], test_pos_edge_label=[10], test_neg_edge_label_index=[2, 10], test_neg_edge_label=[10], overall_edge_index=[2, 50], edge_index=[2, 40], num_nodes=62, num_edges=50, x=[62, 832], edge_attr=[50, 768], node_type=[62], node_layer=[62], edge_type=[50]),
 GraphData(train_edge_mask=[40], test_edge_mask=[10], train_pos_edge_label_index=[2, 40], train_pos_edge_label=[40], train_neg_edge_label_index=[2, 40], train_neg_edge_label=[40], test_pos_edge_label_index=[2, 10], test_pos_edge_label=[10], test_neg_edge_label_index=[2, 10], test_neg_edge_label=[10], overall_edge_index=[2, 50], edge_index=[2, 40], num_nodes=62, num_edges=50, x=[62, 832], edge_attr=[50, 768], node_type=[62], node_layer=[62], edge_type=[50]))

In [ ]:
texts_data = graph_edge_dataset[0].get_link_prediction_texts('type', 'lp')
texts_data['test_pos_edges'][:20], texts_data['test_neg_edges'][:20]

In [6]:
graph_edge_dataset.file_paths[dataset[0].hash]

'datasets/graph_data/eamodelset/edge_data_dist_0_neg_nsr_1_nt_1_sp_1_use_emb_1_ckpt_results_eamodelset_lp_10_att_0_nt_0_checkpoint-177600_test_0.2/9256ec1b11002dd708b621c9cf26616b/data.pkl'

In [ ]:
torch_dataset = graph_edge_dataset.get_torch_dataset()

In [5]:
torch_dataset[0].x.shape, torch_dataset[0].edge_index.shape, torch_dataset[0].edge_attr.shape

(torch.Size([62, 1]), torch.Size([2, 40]), torch.Size([50, 1]))

In [6]:
torch_dataset[0]

GraphData(train_edge_mask=[40], test_edge_mask=[10], train_pos_edge_label_index=[2, 40], train_pos_edge_label=[40], train_neg_edge_label_index=[2, 40], train_neg_edge_label=[40], test_pos_edge_label_index=[2, 10], test_pos_edge_label=[10], test_neg_edge_label_index=[2, 10], test_neg_edge_label=[10], overall_edge_index=[2, 50], edge_index=[2, 40], num_nodes=62, num_edges=50, node_type=[62], node_layer=[62], edge_type=[50], x=[62, 1], edge_attr=[50, 1])

In [ ]:
from tqdm.auto import tqdm
import torch
import torch.nn.functional as F
from models.gnn_layers import *
from torch_geometric.loader import DataLoader
from torch_geometric.nn import SAGEConv, GATv2Conv


class GNNLinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GNNLinkPredictor, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.link_predictor = torch.nn.Sequential(
            torch.nn.Linear(2 * hidden_channels, hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_channels, 2)  # Two output classes: True or False
        )

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

    def decode(self, z, edge_index):
        src, dst = edge_index
        edge_features = torch.cat([z[src], z[dst]], dim=1)
        return self.link_predictor(edge_features)

    def forward(self, x, edge_index, edge_pos):
        z = self.encode(x, edge_index)
        return self.decode(z, edge_pos)



def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()

    # Use the training positive and negative edges
    edge_pos = data.train_pos_edge_label_index
    edge_neg = data.train_neg_edge_label_index

    # Combine positive and negative edges
    edge_index = torch.cat([edge_pos, edge_neg], dim=1)
    labels = torch.cat([torch.ones(edge_pos.size(1)), torch.zeros(edge_neg.size(1))]).long().to(data.x.device)

    # Forward pass
    out = model(data.x, edge_pos, edge_index)
    loss = F.cross_entropy(out, labels)
    loss.backward()
    optimizer.step()
    return loss.item()


@torch.no_grad()
def test(model, data):
    model.eval()

    # Evaluate on test positive and negative edges
    edge_pos = data.test_pos_edge_label_index
    edge_neg = data.test_neg_edge_label_index
    edge_index = torch.cat([edge_pos, edge_neg], dim=1)
    labels = torch.cat([torch.ones(edge_pos.size(1)), torch.zeros(edge_neg.size(1))]).long().to(data.x.device)

    # Forward pass
    out = model(data.x, edge_pos, edge_index)
    pred = out.argmax(dim=1)  # Get predicted class (0 or 1)
    accuracy = (pred == labels).sum().item() / labels.size(0)
    return accuracy


def run(dataloader, num_epochs=100):
	# Training loop
	for epoch in range(1, num_epochs):
		loss = 0
		for data in dataloader:  # Iterate over each graph in the dataset
			data = data.to(device)
			loss += train(model, data, optimizer)
		
		print(f'Epoch {epoch}, Loss: {loss:.4f}')

		# Test loop
		total_accuracy = 0
		for data in dataloader:
			data = data.to(device)
			total_accuracy += test(model, data)
		average_accuracy = total_accuracy / len(dataloader)
		print(f'Average Test Accuracy: {average_accuracy:.4f}')


hidden_channels = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNLinkPredictor(in_channels=torch_dataset[0].x.shape[-1], hidden_channels=hidden_channels).to(device)  # Adjust input size accordingly

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
dataloader = DataLoader(torch_dataset, batch_size=16, shuffle=True)
run(dataloader, num_epochs=20)

Epoch 1, Loss: 20.1870
Average Test Accuracy: 0.8861
Epoch 2, Loss: 19.4412
Average Test Accuracy: 0.8885
Epoch 3, Loss: 19.1393
Average Test Accuracy: 0.8865
Epoch 4, Loss: 19.1695
Average Test Accuracy: 0.8885
Epoch 5, Loss: 19.2538
Average Test Accuracy: 0.8878
Epoch 6, Loss: 19.2151
Average Test Accuracy: 0.8877
Epoch 7, Loss: 19.0947
Average Test Accuracy: 0.8873
Epoch 8, Loss: 19.1740
Average Test Accuracy: 0.8876
Epoch 9, Loss: 19.2315
Average Test Accuracy: 0.8875
Epoch 10, Loss: 19.1580
Average Test Accuracy: 0.8869
Epoch 11, Loss: 19.2266
Average Test Accuracy: 0.8882
Epoch 12, Loss: 19.1967
Average Test Accuracy: 0.8867
Epoch 13, Loss: 19.2607
Average Test Accuracy: 0.8866
Epoch 14, Loss: 19.2285
Average Test Accuracy: 0.8868
Epoch 15, Loss: 19.2100
Average Test Accuracy: 0.8881
Epoch 16, Loss: 19.2172
Average Test Accuracy: 0.8886
Epoch 17, Loss: 19.2189
Average Test Accuracy: 0.8867
Epoch 18, Loss: 19.1475
Average Test Accuracy: 0.8864
Epoch 19, Loss: 19.2123
Average Test 

In [11]:
from collections import defaultdict


edges = defaultdict(int)

for graph in graph_edge_dataset:
	pos_edge_idx = graph.data.edge_index
	train_pos_edge_index = pos_edge_idx
	train_neg_edge_index = graph.data.train_neg_edge_label_index
	test_pos_edge_index = graph.data.test_pos_edge_label_index
	test_neg_edge_index = graph.data.test_neg_edge_label_index

	assert set((a, b) for a, b in train_pos_edge_index.T.tolist()).issubset(set(graph.graph.numbered_graph.edges()))
	assert set((a, b) for a, b in test_pos_edge_index.T.tolist()).issubset(set(graph.graph.numbered_graph.edges()))

	assert len(set(graph.graph.numbered_graph.edges()).intersection(set((a, b) for a, b in test_neg_edge_index.T.tolist()))) == 0
	assert len(set(graph.graph.numbered_graph.edges()).intersection(set((a, b) for a, b in train_neg_edge_index.T.tolist()))) == 0

	assert len(set((a, b) for a, b in train_pos_edge_index.T.tolist()).intersection(set((a, b) for a, b in test_neg_edge_index.T.tolist()))) == 0
	assert len(set((a, b) for a, b in train_pos_edge_index.T.tolist()).intersection(set((a, b) for a, b in test_neg_edge_index.T.tolist()))) == 0

	edges['train_positive'] += train_pos_edge_index.size(1)
	edges['train_negative'] += train_neg_edge_index.size(1)
	edges['test_positive'] += test_pos_edge_index.size(1)
	edges['test_negative'] += test_neg_edge_index.size(1)

edges


defaultdict(int,
            {'train_positive': 56802,
             'train_negative': 56802,
             'test_positive': 13923,
             'test_negative': 13923})

In [ ]:
graph_data_params = dict(
    # reload=True,
    test_ratio=0.2,
    distance=0,
    use_edge_types=True,
    use_special_tokens=True,
    use_embeddings=True,
    embed_model_name='bert-base-cased',
    ckpt='results/eamodelset/edge_cls/type_10_att_0_nt_1/checkpoint-2200'
)

print("Loading graph dataset")
graph_node_dataset = GraphNodeDataset(dataset, **graph_data_params)
print("Loaded graph dataset")

In [ ]:
edge_texts = graph_edge_dataset.get_link_prediction_texts()

In [3]:
graph_torch_data = graph_edge_dataset.get_torch_geometric_data(
    # use_edge_types=True,
)

In [6]:
len(list(dict(graph_node_dataset[0].graph.nodes(data='type')).values()))

15

In [7]:
for torch_graph in graph_torch_data:
    print(torch_graph)
    break

GraphData(train_edge_mask=[34], test_edge_mask=[8], train_pos_edge_label_index=[2, 34], train_pos_edge_label=[34], train_neg_edge_label_index=[2, 34], train_neg_edge_label=[34], test_pos_edge_label_index=[2, 8], test_pos_edge_label=[8], test_neg_edge_label_index=[2, 8], test_neg_edge_label=[8], overall_edge_index=[2, 42], edge_index=[2, 34], x=[97, 768], edge_attr=[42, 768], num_nodes=97, num_edges=42, node_type=[97], node_layer=[97], edge_type=[42])


In [8]:
num_classes = getattr(graph_edge_dataset, 'num_edges_type')
print(num_classes)

11


In [ ]:
import torch

for label in range(num_classes):
    print("Label: ", label)
    for torch_graph in graph_torch_data:
        indices = (torch_graph.edge_type == label).nonzero(as_tuple=True)[0]
        selected_embeddings = torch_graph.edge_attr[indices]
        if len(selected_embeddings) == 0:
            continue
        comparison = torch.all(selected_embeddings == selected_embeddings[0], dim=1)
        if torch.where(~comparison)[0].shape[0] > 0:
            assert torch.where(~comparison)[0].shape[0] == 0, f"Label {label} has different embeddings"

In [10]:
from torch_geometric.loader import DataLoader
import torch
from torch_geometric.data import Data


class GraphData(Data):
    def __inc__(self, key, value, *args, **kwargs):
        if 'index' in key or 'node_mask' in key:
            return self.num_nodes
        elif 'edge_mask' in key:
            return self.num_edges
        else:
            return 0

    def __cat_dim__(self, key, value, *args, **kwargs):
        if 'index' in key:
            return 1
        else:
            return 0

edge_index = torch.tensor([
   [0, 1, 1, 2],
   [1, 0, 2, 1],
])
edge_mask = torch.tensor([1, 0, 1, 0])
label = torch.tensor([0, 1, 2, 5])
foo = torch.randn(16)
data = GraphData(num_nodes=3, edge_index=edge_index, foo=foo, edge_mask=edge_mask, label=label)
data_list = [data, data]
loader = DataLoader(data_list, batch_size=2)
batch = next(iter(loader))
print(batch.label)

tensor([0, 1, 2, 5, 0, 1, 2, 5])


In [18]:
def check_batch(batch, label):
    print("Label: ", label)
    train_mask = batch.train_edge_mask
    test_mask = batch.test_edge_mask

    labels = (batch.edge_type == label).nonzero(as_tuple=True)[0]
    train_indices = labels[torch.isin(labels, train_mask)]
    test_indices = labels[torch.isin(labels, test_mask)]

    train_selected_embeddings = batch.edge_attr[train_indices]
    test_selected_embeddings = batch.edge_attr[test_indices]
    
    if len(train_selected_embeddings) > 0:
        train_cosine_similarity = torch.nn.functional.cosine_similarity(train_selected_embeddings, train_selected_embeddings[0].repeat(len(train_selected_embeddings), 1))
        assert abs(int(sum(train_cosine_similarity)) - len(train_cosine_similarity)) <= 2, f"Train Label {label} has different embeddings"
    
    if len(test_selected_embeddings) > 0:
        test_cosine_similarity = torch.nn.functional.cosine_similarity(test_selected_embeddings, test_selected_embeddings[0].repeat(len(test_selected_embeddings), 1))
        assert abs(int(sum(test_cosine_similarity)) - len(test_cosine_similarity)) <= 2, f"Test Label {label} has different embeddings"


In [16]:
import torch

def intersect_tensors(t1: torch.Tensor, t2: torch.Tensor) -> torch.Tensor:
    # Use torch.isin to get a mask of elements in t1 that are also in t2
    mask = torch.isin(t1, t2)
    
    # Use the mask to filter t1
    t3 = t1[mask]
    
    return t3

# Example usage:
t1 = torch.tensor([1, 2, 3, 4, 5])
t2 = torch.tensor([3, 4, 6, 7, 10, 12, 15, 14])

t3 = intersect_tensors(t1, t2)
print(t3)  # Output: tensor([3, 4, 5])

tensor([3, 4])


In [20]:
for label in range(num_classes):
    check_batch(batch, label)

Label:  0
Label:  1
Label:  2
Label:  3
Label:  4
Label:  5
Label:  6
Label:  7
Label:  8
Label:  9
Label:  10


In [21]:
dataloader = DataLoader(graph_torch_data, batch_size=2)

for batch in dataloader:
    for label in range(num_classes):
        check_batch(batch, label)

Label:  0
Label:  1
Label:  2
Label:  3
Label:  4
Label:  5
Label:  6
Label:  7
Label:  8
Label:  9
Label:  10
Label:  0
Label:  1
Label:  2
Label:  3
Label:  4
Label:  5
Label:  6
Label:  7
Label:  8
Label:  9
Label:  10
Label:  0
Label:  1
Label:  2
Label:  3
Label:  4
Label:  5
Label:  6
Label:  7
Label:  8
Label:  9
Label:  10
Label:  0
Label:  1
Label:  2
Label:  3
Label:  4
Label:  5
Label:  6
Label:  7
Label:  8
Label:  9
Label:  10
Label:  0
Label:  1
Label:  2
Label:  3
Label:  4
Label:  5
Label:  6
Label:  7
Label:  8
Label:  9
Label:  10
Label:  0
Label:  1
Label:  2
Label:  3
Label:  4
Label:  5
Label:  6
Label:  7
Label:  8
Label:  9
Label:  10
Label:  0
Label:  1
Label:  2
Label:  3
Label:  4
Label:  5
Label:  6
Label:  7
Label:  8
Label:  9
Label:  10
Label:  0
Label:  1
Label:  2
Label:  3
Label:  4
Label:  5
Label:  6
Label:  7
Label:  8
Label:  9
Label:  10
Label:  0
Label:  1
Label:  2
Label:  3
Label:  4
Label:  5
Label:  6
Label:  7
Label:  8
Label:  9
Label:  10
L

In [24]:
graph_torch_data[0].edge_type, graph_torch_data[1].edge_type

(tensor([ 2, 10,  1, 10,  3, 10, 10,  9, 10, 10,  5,  2,  5,  8,  0,  8,  5,  0,
          4,  7,  3,  0,  0,  8, 10,  7,  7,  8,  2,  2,  7,  7,  8,  7,  2,  4,
          6, 10, 10,  4,  4, 10]),
 tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 7, 7, 7, 4, 4,
         4, 4, 4, 7, 7, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4]))

In [25]:
graph_torch_data[0].train_edge_mask, graph_torch_data[1].train_edge_mask

(tensor([18,  3, 20, 33,  4,  0, 37, 25, 32,  1, 22,  9,  7, 14, 17,  2, 27, 23,
         31, 39, 15, 24, 13, 12, 35, 40, 16, 29, 26,  5, 36, 34,  8, 38]),
 tensor([44,  7,  9, 37, 31,  6, 39,  8, 14, 13, 29,  5, 27,  1, 46, 19, 45, 23,
         36, 40, 33, 15,  0, 17, 24,  4, 34, 32, 43, 12, 16, 21, 10, 48, 11, 35,
         38, 41,  2,  3]))

In [26]:
graph_torch_data[0].train_edge_mask

tensor([18,  3, 20, 33,  4,  0, 37, 25, 32,  1, 22,  9,  7, 14, 17,  2, 27, 23,
        31, 39, 15, 24, 13, 12, 35, 40, 16, 29, 26,  5, 36, 34,  8, 38])

In [27]:
graph_torch_data[0].edge_type[graph_torch_data[0].train_edge_mask]

tensor([ 4, 10,  3,  7,  3,  2, 10,  7,  8, 10,  0, 10,  9,  0,  0,  1,  8,  8,
         7,  4,  8, 10,  8,  5,  4,  4,  5,  2,  7, 10,  6,  2, 10, 10])

In [28]:
graph_torch_data[0].edge_type[graph_torch_data[0].train_edge_mask] 

tensor([ 4, 10,  3,  7,  3,  2, 10,  7,  8, 10,  0, 10,  9,  0,  0,  1,  8,  8,
         7,  4,  8, 10,  8,  5,  4,  4,  5,  2,  7, 10,  6,  2, 10, 10])

In [29]:
g1_ind, g2_ind = (graph_torch_data[0].edge_type[graph_torch_data[0].train_edge_mask] == 4).nonzero(as_tuple=True)[0], (graph_torch_data[1].edge_type[graph_torch_data[1].train_edge_mask] == 4).nonzero(as_tuple=True)[0]
g1_ind, g2_ind

(tensor([ 0, 19, 24, 25]),
 tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 13, 14, 16, 17, 18, 19, 20,
         21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39]))

In [30]:
batch.train_edge_mask

tensor([18,  3, 20, 33,  4,  0, 37, 25, 32,  1, 22,  9,  7, 14, 17,  2, 27, 23,
        31, 39, 15, 24, 13, 12, 35, 40, 16, 29, 26,  5, 36, 34,  8, 38, 86, 49,
        51, 79, 73, 48, 81, 50, 56, 55, 71, 47, 69, 43, 88, 61, 87, 65, 78, 82,
        75, 57, 42, 59, 66, 46, 76, 74, 85, 54, 58, 63, 52, 90, 53, 77, 80, 83,
        44, 45])

In [47]:
batch_indices = (batch.edge_type[batch.train_edge_mask] == 4).nonzero(as_tuple=True)[0]
batch_indices

tensor([ 0, 19, 24, 25, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 47, 48, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69,
        70, 71, 72, 73])

In [31]:
batch.edge_type[batch.train_edge_mask]

tensor([ 4, 10,  3,  7,  3,  2, 10,  7,  8, 10,  0, 10,  9,  0,  0,  1,  8,  8,
         7,  4,  8, 10,  8,  5,  4,  4,  5,  2,  7, 10,  6,  2, 10, 10,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  8,  4,  7,  4,  4,  7,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  7,  4,  4,  4,  4,  4,  4,
         4,  4])

In [33]:
sum(batch.edge_attr[batch.train_edge_mask][1] == batch.edge_attr[batch.train_edge_mask][6])

tensor(768)

In [130]:
batch.num_nodes, graph_torch_data[0].num_nodes

(358, 261)

In [127]:
graph_torch_data[0].train_edge_mask[:20]

tensor([ 60, 358, 295, 149, 321, 181, 154,  62, 260, 345, 381, 394, 101, 264,
        378, 361, 288,  89, 122, 175])

In [128]:
graph_torch_data[1].train_edge_mask[:20]

tensor([38, 33, 20, 21, 19, 18, 26, 24,  1,  7, 29, 36, 12, 37,  6, 11, 41,  5,
        14,  9])

In [131]:
batch.train_edge_mask[261:281]

tensor([291,  37, 121, 372, 240, 327, 362,  17,   9,  29, 179, 271, 371, 254,
          7, 123,  70,  47, 162, 302])

In [ ]:
num_classes = getattr(graph_edge_dataset, 'num_edges_type')
for batch in dataloader:
    check_batch(batch, num_classes)

In [55]:
batch.edge_type[146]

tensor(10)

In [71]:
indices = (batch.edge_type == 3).nonzero(as_tuple=True)[0]
indices.shape

torch.Size([483])

In [74]:
indices[0], indices[246]

(tensor(28), tensor(874))

In [65]:
batch.edge_type[655]

tensor(8)

In [75]:
batch.edge_attr[indices[0]][:20], batch.edge_attr[indices[246]][:20]

(tensor([-0.8145, -0.0470, -0.2188,  0.0849, -0.0718, -0.0343,  0.3368, -0.5850,
          0.0766, -0.4512, -1.1710,  0.5025,  0.0528,  0.0865, -0.1269,  0.4233,
          1.0132,  0.6537,  0.2685, -1.4494]),
 tensor([-0.8144, -0.0473, -0.2187,  0.0840, -0.0722, -0.0341,  0.3368, -0.5854,
          0.0773, -0.4516, -1.1710,  0.5032,  0.0532,  0.0867, -0.1271,  0.4232,
          1.0130,  0.6537,  0.2690, -1.4496]))

In [72]:
selected_embeddings = batch.edge_attr[indices]
selected_embeddings.shape

torch.Size([483, 768])

In [73]:
import torch
comparison = torch.all(selected_embeddings == selected_embeddings[0], dim=1)
torch.where(~comparison)[0]

tensor([246, 247, 248, 249, 371, 372, 373])

In [ ]:
getattr(graph_node_dataset, 'num_nodes_type')

In [4]:
getattr(graph_node_dataset, f"node_exclude_type")

[63]

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network model
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(SimpleClassifier, self).__init__()
        
        # Define layers
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layer3 = nn.Linear(hidden_size2, output_size)
        
        # Define activation and dropout
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        # Pass data through the first layer, apply activation and dropout
        # x = torch.cat([x[edge_index[0]], x[edge_index[1]]], dim=1)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # Second layer
        x = self.layer2(x)
        x = self.relu(x)
        x = self.dropout(x)

        # Third layer (output layer)
        x = self.layer3(x)
        
        return x

# Example usage with data X, y
exclude_labels = getattr(graph_edge_dataset, f"edge_exclude_type")
input_size = graph_torch_data[0].edge_attr.shape[1]  # Assuming X is of shape (batch_size, input_features)
hidden_size1 = 128
hidden_size2 = 64
output_size = getattr(graph_edge_dataset, 'num_edges_type')  # Assuming y contains class labels, this will be the number of classes

# Initialize the model, loss function, and optimizer
model = SimpleClassifier(input_size, hidden_size1, hidden_size2, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example training loop
num_epochs = 100
for epoch in range(num_epochs):
    for batch in dataloader:
        train_mask = batch.train_edge_mask
        model.train()
        
        # Forward pass
        outputs = model(batch.edge_attr[train_mask])
        # pred = outputs[train_mask]
        labels = batch.edge_type[train_mask]
        
        mask = ~torch.isin(labels, torch.tensor(exclude_labels))

        loss = criterion(outputs[mask], labels[mask])
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/100], Loss: 2.116132974624634
Epoch [2/100], Loss: 1.8642759323120117
Epoch [3/100], Loss: 0.9200063347816467
Epoch [4/100], Loss: 1.4972515106201172
Epoch [5/100], Loss: 1.2084574699401855
Epoch [6/100], Loss: 0.6340409517288208
Epoch [7/100], Loss: 0.5120918154716492
Epoch [8/100], Loss: 0.3598491847515106
Epoch [9/100], Loss: 0.46989649534225464
Epoch [10/100], Loss: 0.46171605587005615
Epoch [11/100], Loss: 0.23111966252326965
Epoch [12/100], Loss: 0.05078734830021858
Epoch [13/100], Loss: 0.03232596442103386
Epoch [14/100], Loss: 0.103300541639328
Epoch [15/100], Loss: 0.04028802365064621
Epoch [16/100], Loss: 0.07596289366483688
Epoch [17/100], Loss: 0.06882505863904953
Epoch [18/100], Loss: 0.07486800849437714
Epoch [19/100], Loss: 0.024326512590050697
Epoch [20/100], Loss: 0.009404868818819523
Epoch [21/100], Loss: 0.18503651022911072
Epoch [22/100], Loss: 0.058016229420900345
Epoch [23/100], Loss: 0.03199069947004318
Epoch [24/100], Loss: 0.01217227429151535
Epoch [25

In [61]:
graph_torch_data[0].edge_type

tensor([ 2, 10,  1, 10,  3, 10, 10,  9, 10, 10,  5,  2,  5,  8,  0,  8,  5,  0,
         4,  7,  3,  0,  0,  8, 10,  7,  7,  8,  2,  2,  7,  7,  8,  7,  2,  4,
         6, 10, 10,  4,  4, 10])

In [66]:
sum(graph_torch_data[0].edge_attr[0] == graph_torch_data[0].edge_attr[11])

tensor(768)

In [4]:
import torch
from torch.nn import functional as F
from torch_geometric.nn import aggr
from torch_geometric.nn import (
    global_add_pool,
    global_max_pool,
    global_mean_pool,
)
import torch_geometric
import torch.nn as nn


aggregation_methods = {
    'mean': aggr.MeanAggregation(),
    'sum': aggr.SumAggregation(),
    'max': aggr.MaxAggregation(),
    'mul': aggr.MulAggregation(),
}

supported_conv_models = {
    'GCNConv': False, ## True or False if the model requires num_heads
    'GraphConv': False,
    'GATConv': True,
    'SAGEConv': False,
    'GINConv': False,
    'GATv2Conv': True,
}

global_pooling_methods = {
    'sum': global_add_pool,
    'mean': global_mean_pool,
    'max': global_max_pool,
}


class GNNConv(torch.nn.Module):
    """
        A general GNN model created using the PyTorch Geometric library
        model_name: the name of the GNN model
        input_dim: the input dimension
        hidden_dim: the hidden dimension
        out_dim: the output dimension

        num_layers: the number of GNN layers
        num_heads: the number of heads in the GNN layer
        residual: whether to use residual connections
        l_norm: whether to use layer normalization
        dropout: the dropout probability
    
    """
    def __init__(
            self, 
            model_name, 
            input_dim, 
            hidden_dim, 
            out_dim, 
            num_layers, 
            num_heads=None, 
            residual=False, 
            l_norm=False, 
            dropout=0.1,
            aggregation='mean',
            edge_dim=None
        ):
        super(GNNConv, self).__init__()

        assert model_name in supported_conv_models, f"Model {model_name} not supported. Choose from {supported_conv_models.keys()}"
        heads_supported = supported_conv_models[model_name]
        if heads_supported and num_heads is None:
            raise ValueError(f"Model {model_name} requires num_heads to be set to an integer")
        
        if not heads_supported and num_heads is not None:
            num_heads = None

        assert aggregation in aggregation_methods, f"Aggregation method {aggregation} not supported. Choose from {aggregation_methods.keys()}"
        aggregation = aggregation_methods[aggregation]

        self.input_dim = input_dim
        self.embed_dim = hidden_dim
        self.out_dim = out_dim
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.residual = residual
        self.l_norm = l_norm
        self.dropout = dropout
        self.aggregation = aggregation
        self.edge_dim = edge_dim
        

        gnn_model = getattr(torch_geometric.nn, model_name)
        self.conv_layers = nn.ModuleList()
        if num_heads is None:
            input_layer = gnn_model(input_dim, hidden_dim, aggr=aggregation)
        else:
            input_layer = gnn_model(
                input_dim, 
                hidden_dim, 
                heads=num_heads, 
                aggr=aggregation,
                edge_dim=edge_dim
            )
        self.conv_layers.append(input_layer)

        for _ in range(num_layers - 2):
            if num_heads is None:
                conv = gnn_model(hidden_dim, hidden_dim, aggr=aggregation)
                self.conv_layers.append(conv)
            else:
                conv = gnn_model(
                    num_heads*hidden_dim, 
                    hidden_dim, 
                    heads=num_heads, 
                    aggr=aggregation,
                    edge_dim=edge_dim
                )
                self.conv_layers.append(conv)

        if num_heads is None:
            self.conv_layers.append(gnn_model(hidden_dim, out_dim, aggr=aggregation))
        else:
            self.conv_layers.append(gnn_model(
                num_heads*hidden_dim, 
                out_dim, 
                heads=num_heads, 
                aggr=aggregation,
                edge_dim=edge_dim
            ))
            
        self.activation = nn.ReLU()
        self.layer_norm = nn.LayerNorm(hidden_dim if num_heads is None else num_heads*hidden_dim) if l_norm else None
        self.residual = residual
        self.dropout = nn.Dropout(dropout)


    def forward(self, in_feat, edge_index, edge_attr=None):
        
        h = in_feat
        h = self.conv_layers[0](h, edge_index, edge_attr) if isinstance(edge_attr, torch.Tensor) else self.conv_layers[0](h, edge_index)
        h = self.activation(h)
        if self.layer_norm is not None:
            h = self.layer_norm(h)
        h = self.dropout(h)

        for conv in self.conv_layers[1:-1]:
            nh = conv(h, edge_index, edge_attr) if edge_attr is not None else conv(h, edge_index)
            h = nh if not self.residual else nh + h
            h = self.activation(h)
            if self.layer_norm is not None:
                h = self.layer_norm(h)
            h = self.dropout(h)
        
        h = self.conv_layers[-1](h, edge_index)
        h = self.activation(h)
        if self.layer_norm is not None:
            h = self.layer_norm(h)
        h = self.dropout(h)
        return h
  

class EdgeClassifer(nn.Module):

    """
    An MLP predictor for link prediction

    h_feats: the input dimension
    num_classes: the number of classes
    num_layers: the number of layers in the MLP

    This class concatenates the node embeddings of the two nodes in the edge
    The concatenated embeddings are then passed through an MLP
    """

    def __init__(
            self, 
            input_dim,
            hidden_dim, 
            num_classes,
            num_layers=2, 
            dropout=0.1,
            edge_dim=None,
            bias=True
        ):
        super().__init__()
        self.layers = nn.ModuleList()
        self.input_dim = input_dim
        self.embed_dim = hidden_dim
        self.num_layers = num_layers
        self.num_classes = num_classes

        # in_feats = input_dim * 2
        # if edge_dim is not None:
        #     in_feats += edge_dim
        
        in_feats = edge_dim
        
        for _ in range(num_layers):
            self.layers.append(nn.Linear(in_feats, hidden_dim, bias=bias))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.Dropout(dropout))
            in_feats = hidden_dim
        
        self.layers.append(nn.Linear(hidden_dim, num_classes, bias=bias))
        self.layers.append(nn.ReLU())


    def forward(self, x, edge_index, edge_attr=None):
        # h = torch.cat([x[edge_index[0]], x[edge_index[1]]], dim=-1)
        # if edge_attr is not None:
        #     h = torch.cat([h, edge_attr], dim=-1)
        
        h = edge_attr
        for layer in self.layers:
            h = layer(h)
        
        return h
    

class NodeClassifier(nn.Module):

    """
    An MLP predictor for link prediction

    h_feats: the input dimension
    num_classes: the number of classes
    num_layers: the number of layers in the MLP

    This class concatenates the node embeddings of the two nodes in the edge
    The concatenated embeddings are then passed through an MLP
    """

    def __init__(
            self, 
            input_dim,
            hidden_dim, 
            num_classes,
            num_layers=2, 
            dropout=0.3,
            bias=True
        ):
        super().__init__()
        self.layers = nn.ModuleList()
        self.embed_dim = hidden_dim
        self.num_layers = num_layers
        self.num_classes = num_classes

        for _ in range(num_layers - 1):
            self.layers.append(nn.Linear(input_dim, hidden_dim, bias=bias))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.Dropout(dropout))
            input_dim = hidden_dim
        
        self.layers.append(nn.Linear(hidden_dim, num_classes, bias=bias))
        self.layers.append(nn.ReLU())


    def forward(self, x):
        h = x
        for layer in self.layers:
            h = layer(h)
        
        return h
    

class GraphClassifer(nn.Module):

    """
    An MLP predictor for link prediction

    h_feats: the input dimension
    num_classes: the number of classes
    num_layers: the number of layers in the MLP

    This class concatenates the node embeddings of the two nodes in the edge
    The concatenated embeddings are then passed through an MLP
    """

    def __init__(
            self, 
            input_dim, 
            num_classes,
            global_pool='mean',
            bias=False
        ):
        super().__init__()
        self.layers = nn.ModuleList()
        self.input_dim = input_dim
        self.num_classes = num_classes

        self.layers.append(nn.Linear(input_dim, num_classes, bias=bias))
        self.global_pool = global_pooling_methods[global_pool]

    def forward(self, x, batch):
        h = self.global_pool(x, batch)
        for layer in self.layers:
            h = layer(h)
        
        return h


In [5]:
import os
input_dim = graph_torch_data[0].x.shape[1]


model_name = 'GATv2Conv'

hidden_dim = 128
output_dim = 128
num_conv_layers = 3
num_mlp_layers = 1
num_heads = 2
residual = False
l_norm = False
dropout = 0.1
aggregation = 'mean'

num_edges_label = f"num_edges_type"
num_nodes_label = f"num_nodes_type"
assert hasattr(graph_edge_dataset, num_edges_label), f"Graph dataset does not have attribute {num_edges_label}"
num_edge_classes = getattr(graph_edge_dataset, num_edges_label)
# assert hasattr(graph_node_dataset, num_nodes_label), f"Graph dataset does not have attribute {num_nodes_label}"
# num_node_classes = getattr(graph_node_dataset, num_nodes_label)


edge_dim = graph_edge_dataset[0].data.edge_attr.shape[1]

logs_dir = os.path.join("logs")
use_edge_attrs = True

gnn_conv_model = GNNConv(
    model_name=model_name,
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    out_dim=output_dim,
    num_layers=num_conv_layers,
    num_heads=num_heads,
    residual=residual,
    l_norm=l_norm,
    dropout=dropout,
    aggregation=aggregation,
    edge_dim=edge_dim
)

clf_input_dim = output_dim
edge_mlp_predictor = EdgeClassifer(
    input_dim=clf_input_dim if not num_heads else num_heads*clf_input_dim,
    hidden_dim=hidden_dim,
    num_layers=num_mlp_layers, 
    num_classes=num_edge_classes,
    edge_dim=edge_dim if use_edge_attrs else None,
    bias=False,
)

node_mlp_predictor = NodeClassifier(
    input_dim=clf_input_dim,
    hidden_dim=hidden_dim,
    num_layers=num_mlp_layers, 
    num_classes=12,
    bias=False,
)

def init_weights(m):
    if isinstance(m, torch.nn.Linear) or isinstance(m, torch.nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)

gnn_conv_model, edge_mlp_predictor, node_mlp_predictor

(GNNConv(
   (aggregation): MeanAggregation()
   (conv_layers): ModuleList(
     (0): GATv2Conv(768, 128, heads=2)
     (1-2): 2 x GATv2Conv(256, 128, heads=2)
   )
   (activation): ReLU()
   (dropout): Dropout(p=0.1, inplace=False)
 ),
 EdgeClassifer(
   (layers): ModuleList(
     (0): Linear(in_features=768, out_features=128, bias=False)
     (1): ReLU()
     (2): Dropout(p=0.1, inplace=False)
     (3): Linear(in_features=128, out_features=11, bias=False)
     (4): ReLU()
   )
 ),
 NodeClassifier(
   (layers): ModuleList(
     (0): Linear(in_features=128, out_features=12, bias=False)
     (1): ReLU()
   )
 ))

In [6]:
from settings import device
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from itertools import chain
import torch.nn as nn


gnn_conv_model.apply(init_weights)
node_mlp_predictor.apply(init_weights)
edge_mlp_predictor.apply(init_weights)

cls_label = 'type'
num_epochs = 300

optimizer = Adam(chain(gnn_conv_model.parameters(), node_mlp_predictor.parameters()), lr=1e-3)
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)

edge2index = lambda g: torch.stack(list(g.edges())).contiguous()
criterion = nn.CrossEntropyLoss()

In [8]:
from torch_geometric.loader import DataLoader

dataloader = DataLoader(
    graph_torch_data, 
    batch_size=16, 
    shuffle=True
)

In [57]:
torch.autograd.set_detect_anomaly(True)

In [9]:
from collections import defaultdict

from sklearn.metrics import f1_score, balanced_accuracy_score, precision_score, recall_score
import torch


def compute_classification_metrics(preds, labels):
    """
    Compute F1-score, balanced accuracy, precision, and recall for multi-class classification.
    
    Args:
        preds (torch.Tensor): Predictions from the model (logits or probabilities). Shape: [num_samples, num_classes]
        labels (torch.Tensor): Ground truth labels. Shape: [num_samples]
    
    Returns:
        dict: Dictionary containing metrics (F1-score, balanced accuracy, precision, recall).
    """
    # Convert predictions to class labels
    preds = torch.argmax(preds, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()

    metrics = {}

    # F1-score (macro and weighted)
    metrics['f1_macro'] = f1_score(labels, preds, average='macro')
    metrics['f1_weighted'] = f1_score(labels, preds, average='weighted')

    # Balanced Accuracy
    metrics['balanced_accuracy'] = balanced_accuracy_score(labels, preds)

    # Precision (macro and weighted)
    metrics['precision_macro'] = precision_score(labels, preds, average='macro', zero_division=0)
    metrics['precision_weighted'] = precision_score(labels, preds, average='weighted', zero_division=0)

    # Recall (macro and weighted)
    metrics['recall_macro'] = recall_score(labels, preds, average='macro', zero_division=0)
    metrics['recall_weighted'] = recall_score(labels, preds, average='weighted', zero_division=0)

    return metrics


def get_logits(model, x, edge_index, edge_attr=None):
    if edge_attr is not None:
        edge_attr = edge_attr.to(device)
        h = model(
            x.to(device), 
            edge_index.to(device), 
            edge_attr.to(device)
        )
    else:
        h = model(
            x.to(device), 
            edge_index.to(device)
        )
    return h


def get_prediction_score(predictor, h, edge_index=None, edge_attr=None):
    if edge_attr is not None:
        prediction_score = predictor(
            h.to(device), 
            edge_index.to(device), 
            edge_attr.to(device)
        )
    elif edge_index is not None and not isinstance(predictor, NodeClassifier):
        prediction_score = predictor(
            h.to(device), 
            edge_index.to(device)
        )
    else:
        prediction_score = predictor(h.to(device))
    return prediction_score


def train(model, predictor):
    exclude_labels = getattr(graph_edge_dataset, f"edge_exclude_type")
    model.train()
    predictor.train()
    all_preds, all_labels = list(), list()
    epoch_loss = 0
    epoch_metrics = defaultdict(float)
    for data in dataloader:
        optimizer.zero_grad()
        gnn_conv_model.zero_grad()
        predictor.zero_grad()
        x = data.x
        edge_index =  data.edge_index
        train_mask = data.train_edge_mask
        edge_attr = data.edge_attr[train_mask] if use_edge_attrs else None
        
        h = get_logits(model, x, edge_index, edge_attr)
        scores = get_prediction_score(predictor, h, edge_index, edge_attr)[train_mask]
        labels = getattr(data, f"edge_{cls_label}")[train_mask]

        mask = ~torch.isin(labels, torch.tensor(exclude_labels))
        loss = criterion(scores[mask], labels[mask].to(device))
        all_preds.append(scores[mask].detach().cpu())
        all_labels.append(labels[mask])

        loss.backward()
        optimizer.step()
        scheduler.step()
        epoch_loss += loss.item()
    

    
    all_preds = torch.cat(all_preds, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    epoch_metrics = compute_classification_metrics(all_preds, all_labels)
    epoch_metrics['loss'] = epoch_loss        
    epoch_metrics['phase'] = 'train'

    # print(f"Train Loss: {epoch_loss}")
    print(f"Train Metrics: {epoch_metrics}")

    return epoch_metrics


def test(model, predictor):
    exclude_labels = getattr(graph_edge_dataset, f"node_exclude_type")
    model.eval()
    predictor.eval()
    all_preds, all_labels = list(), list()
    with torch.no_grad():
        epoch_loss = 0
        epoch_metrics = defaultdict(float)
        for data in dataloader:
            x = data.x
            edge_index =  data.edge_index
            test_mask = data.test_edge_mask
            train_mask = data.train_edge_mask
            edge_attr = data.edge_attr[train_mask] if use_edge_attrs else None
            
            h = get_logits(model, x, edge_index, edge_attr)

            scores = get_prediction_score(predictor, h, edge_index, edge_attr)[test_mask]
            labels = getattr(data, f"edge_{cls_label}")[test_mask]

            mask = ~torch.isin(labels, torch.tensor(exclude_labels))

            all_preds.append(scores[mask].detach().cpu())
            all_labels.append(labels[mask])
            loss = criterion(scores[mask], labels[mask].to(device))
            
            epoch_loss += loss.item()

        all_preds = torch.cat(all_preds, dim=0)
        all_labels = torch.cat(all_labels, dim=0)
        epoch_metrics = compute_classification_metrics(all_preds, all_labels)
        
        epoch_metrics['loss'] = epoch_loss
        epoch_metrics['phase'] = 'test'
        # print(f"Epoch Test Loss: {epoch_loss}\nTest Accuracy: {epoch_acc}\nTest F1: {epoch_f1}")

        # print(f"Test Epoch: {epoch_metrics}")
    
    return epoch_metrics

In [11]:
device = 'cpu'
gnn_conv_model.to(device)
node_mlp_predictor.to(device)
edge_mlp_predictor.to(device)

EdgeClassifer(
  (layers): ModuleList(
    (0): Linear(in_features=768, out_features=128, bias=False)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=128, out_features=11, bias=False)
    (4): ReLU()
  )
)

In [12]:
from tqdm.auto import tqdm


def run_epochs(model, predictor, num_epochs):
    for epoch in tqdm(range(num_epochs), desc="Running Epochs"):
        train_metrics = train(model, predictor)
        test_metrics = test(model, predictor)
        

run_epochs(gnn_conv_model, edge_mlp_predictor, 50)

Running Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

IndexError: index 1138 is out of bounds for dimension 0 with size 1104

In [52]:
import numpy as np

class GNNModel(torch.nn.Module):
    """GraphSage Network"""
    def __init__(
        self, 
        model_name, 
        input_dim, 
        hidden_dim, 
        out_dim, 
        num_layers, 
        num_heads=None, 
        residual=False, 
        l_norm=False, 
        dropout=0.1
    ):
        super(GNNModel, self).__init__()
        gnn_model = getattr(torch_geometric.nn, model_name)
        self.conv_layers = nn.ModuleList()
        if model_name == 'GINConv':
            input_layer = gnn_model(nn.Sequential(nn.Linear(input_dim, hidden_dim), nn.ReLU()), train_eps=True)
        elif num_heads is None:
            input_layer = gnn_model(input_dim, hidden_dim, aggr='SumAggregation')
        else:
            input_layer = gnn_model(input_dim, hidden_dim, heads=num_heads, aggr='SumAggregation')
        self.conv_layers.append(input_layer)

        for _ in range(num_layers - 2):
            if model_name == 'GINConv':
                self.conv_layers.append(gnn_model(nn.Sequential(nn.Linear(hidden_dim, hidden_dim), nn.ReLU()), train_eps=True))
            elif num_heads is None:
                self.conv_layers.append(gnn_model(hidden_dim, hidden_dim, aggr='SumAggregation'))
            else:
                self.conv_layers.append(gnn_model(num_heads*hidden_dim, hidden_dim, heads=num_heads, aggr='SumAggregation'))

        if model_name == 'GINConv':
            self.conv_layers.append(gnn_model(nn.Sequential(nn.Linear(hidden_dim, out_dim), nn.ReLU()), train_eps=True))
        else:
            self.conv_layers.append(gnn_model(hidden_dim if num_heads is None else num_heads*hidden_dim, out_dim, aggr='SumAggregation'))
            
        self.activation = nn.ReLU()
        self.layer_norm = nn.LayerNorm(hidden_dim if num_heads is None else num_heads*hidden_dim) if l_norm else None
        self.residual = residual
        self.dropout = nn.Dropout(dropout)


    def forward(self, in_feat, edge_index, edge_attr=None):
        def ff(layer, h, edge_index, edge_attr):
            if isinstance(edge_attr, torch.Tensor):
                return layer(h, edge_index, edge_attr)
            return layer(h, edge_index)
        
        h = in_feat
        h = ff(self.conv_layers[0], h, edge_index, edge_attr)
        h = self.activation(h)
        if self.layer_norm is not None:
            h = self.layer_norm(h)
        h = self.dropout(h)

        for conv in self.conv_layers[1:-1]:
            h = ff(conv, h, edge_index, edge_attr) if not self.residual else ff(conv, h, edge_index, edge_attr) + h
            h = self.activation(h)
            if self.layer_norm is not None:
                h = self.layer_norm(h)
            h = self.dropout(h)
        
        h = ff(self.conv_layers[-1], h, edge_index, edge_attr)
        return h

gnn_model = GNNModel(
    model_name=model_name,
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    out_dim=output_dim,
    num_layers=num_conv_layers,
    num_heads=num_heads,
    residual=residual,
    l_norm=l_norm,
    dropout=dropout
)

gnn_model

GNNModel(
  (conv_layers): ModuleList(
    (0): GATv2Conv(768, 128, heads=2)
    (1): GATv2Conv(256, 128, heads=2)
    (2): GATv2Conv(256, 128, heads=1)
  )
  (activation): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
class GraphTrainer:
    def __init__(self, model, args) -> None:
        self.model = model
        self.model.to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=args.gnn_lr)
        # self.scheduler = get_linear_schedule_with_warmup(
        #     self.optimizer, num_warmup_steps=10, num_training_steps=args.num_training_steps)
        self.criteria = nn.CrossEntropyLoss()
        self.edge2index = lambda g: torch.stack(list(g.edges())).contiguous()
        self.args = args
        print("GenericGraphTrainer initialized.")

    def run_epoch(self, dataloader, training=True):
        torch.set_grad_enabled(training)
        if training:
            self.model.train()
        else:
            self.model.eval()

        train_preds, train_acts = list(), list()
        test_preds, test_acts = list(), list()
        train_avg_loss, test_avg_loss = 0, 0
        _, topk_test_preds = list(), list()

        # for batch in tqdm(dataloader, total=len(dataloader), desc=f"{train_str} GNN"):
        for batch in dataloader:
            if training:
                self.optimizer.zero_grad()
                self.model.zero_grad()
            # input_ids, labels = batch
            if isinstance(batch, tuple):
                batch = batch[0]

            labels = batch.ndata['labels'].to(device)
            train_mask = batch.ndata['train_mask']
            test_mask = batch.ndata['test_mask']

            # Forward
            # logits = model(batched_graph, features)
            logits = self.get_logits(batch)

            # Compute prediction
            pred = logits.argmax(1)
            if logits[train_mask].shape[0] == 0:
                continue

            # Compute loss for training set nodes
            loss = self.criteria(logits[train_mask], labels[train_mask])
            train_avg_loss += loss.item()
            train_preds += pred[train_mask].tolist()
            train_acts += labels[train_mask].tolist()

            topk_test_preds += logits[test_mask].topk(min(8, logits.shape[-1])).indices.tolist()

            # Backward
            if training:
                loss.backward(torch.ones_like(loss))
                self.optimizer.step()
                # self.scheduler.step()

            if logits[test_mask].shape[0] == 0:
                continue
            # Compute loss for test set nodes
            test_loss = self.criteria(logits[test_mask], labels[test_mask])
            test_avg_loss += test_loss.item()
            test_preds += pred[test_mask].tolist()
            test_acts += labels[test_mask].tolist()

        
        train_acc = (np.array(train_preds) == np.array(train_acts)).mean()
        test_acc = (np.array(test_preds) == np.array(test_acts)).mean()

        # train_predictions_distribution = data_utils.get_predictions_distribution(train_preds, train_acts)
        # test_predictions_distribution = data_utils.get_predictions_distribution(test_preds, test_acts)

        train_avg_loss = np.log2(1 + train_avg_loss)
        test_avg_loss = np.log2(1 + test_avg_loss)
        torch.set_grad_enabled(True)
        output = {
            'train_acc': train_acc, 
            'train_loss': train_avg_loss,
            'test_acc': test_acc,
            'test_loss': test_avg_loss,
            # 'train_predictions_distribution': train_predictions_distribution,
            # 'test_predictions_distribution': test_predictions_distribution
            'top2acc': np.mean([1 if label in topk[:2] else 0 for topk, label in zip(topk_test_preds, test_acts)]),
            'top3acc': np.mean([1 if label in topk[:3] else 0 for topk, label in zip(topk_test_preds, test_acts)]),
            'top4acc': np.mean([1 if label in topk[:4] else 0 for topk, label in zip(topk_test_preds, test_acts)]),
            'top5acc': np.mean([1 if label in topk[:5] else 0 for topk, label in zip(topk_test_preds, test_acts)]),
            'top6acc': np.mean([1 if label in topk[:6] else 0 for topk, label in zip(topk_test_preds, test_acts)]),
        }

        return output

    def get_logits(self, batched_graph):
        self.model.zero_grad()
        x = batched_graph.ndata['feat'].to(device)
        model_name = type(self.model).__name__
        if "GNN" in model_name:
            edge_index = self.edge2index(batched_graph).to(device)
            x = x.float()
            logits = self.model(x, edge_index)
        else:
            logits = self.model(x)
        return logits


    def run_epochs(self, dataloader, num_epochs):
        max_val_acc, max_train_acc = 0, 0
        outputs = list()
        for _ in tqdm(range(num_epochs), desc="Epochs"):
        # for epoch in range(num_epochs):
            output = self.run_epoch(dataloader, training=True)
            
            train_acc, test_acc = output['train_acc'], output['test_acc']
            max_val_acc = max(max_val_acc, test_acc)
            max_train_acc = max(max_train_acc, train_acc)

            # if epoch % 50 == 0 and epoch > 0:
            # print(f"Epoch {epoch}: Train loss {output['train_loss']} and Test Loss {output['test_loss']}")
            # print(f"Train Accuracy: {train_acc} Test Accuracy: {test_acc}")
            outputs.append(output)
        
        print(f"Max Test Accuracy: {max_val_acc}")
        print(f"Max Train Accuracy: {max_train_acc}")
        max_output = max(outputs, key=lambda x: x['test_acc'])
        return max_output
        

    def validate(self, dataloader):
        output = self.run_epoch(dataloader, training=False)
        train_acc, test_acc = output['train_acc'], output['test_acc']
        print(f"Train loss {output['train_loss']} and Test Loss {output['test_loss']}")
        print(f"Train Accuracy: {train_acc} Test Accuracy: {test_acc}")
        return output
    
